In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.models.keyedvectors as word2vec
import gc
import string
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Using TensorFlow backend.


In [3]:
#df = pd.read_csv('Reviews.csv')

In [4]:
def split_sentence(sent):
    sentences = sent_tokenize(sent)
    for i in reversed(range(len(sentences))):
        se = sentences[i]
        sentences[i] = se.strip()
        if se == '':
            sentences.pop(i)
    return(sentences)

In [12]:
def loadEmbeddingMatrix(typeToLoad):
        #load different embedding file from Kaggle depending on which embedding 
        #matrix we are going to experiment with
        if(typeToLoad=="glove"):
            EMBEDDING_FILE='glove.twitter.27B.25d.txt'
            embed_size = 25
        if (typeToLoad=="wiki"):
            EMBEDDING_FILE='wiki-news-300d-1M.vec'
            embed_size = 300
        


        if(typeToLoad=="glove" or typeToLoad =="wiki"):
            embeddings_index = dict()
            #Transfer the embedding weights into a dictionary by iterating through every line of the file.
            f = open(EMBEDDING_FILE, encoding='utf-8')
            for line in f:
                #split up line into an indexed array
                values = line.split()
                #first index is word
                word = values[0]
                #store the rest of the values in the array as a new array
                coefs = np.asarray(values[1:], dtype=float)
                embeddings_index[word] = coefs #50 dimensions
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index))
       # else:
        #    embeddings_index = dict()
         #   for word in word2vecDict.wv.vocab:
          #      embeddings_index[word] = word2vecDict.word_vec(word)
           # print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        return embeddings_index #, embedding_matrix

In [13]:
%time emb_index= loadEmbeddingMatrix('glove')

Loaded 1193515 word vectors.
Wall time: 14.7 s


In [14]:
text = """In the production of SMC (Sheet Moulding Compound), the maturing of the semi-finished product (resin+glass fibre) is of decisive importance. The associated thickening of the material determines the viscosity and thus the quality of the end product. Possible defects due to short maturing and soft semi-finished products are lack of fibre transport, while too long maturing and hard semi-finished products result in incompletely filled components. By adjusting the press parameters such as closing force, closing speed, mould temperature etc., the fluctuations in thickening can normally be compensated. By measuring the flowability/viscosity of the material or by measuring additional parameters during the manufacturing process, the ideal process window for the production of SMC is to be controlled even better."""

Keras:

In [15]:
list_of_sentence = split_sentence(text)
max_feature = 5000
tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(list_of_sentence)
list_of_token_train = tokenizer.texts_to_sequences(list_of_sentence)
maxlen = 300
x_t = pad_sequences(list_of_token_train,maxlen=maxlen)

In [16]:
def embadding_fr_sentence(sentence):
    sent_emb = []
    len_sent = len(sentence)
    for i in sentence:
        i = i.lower()
        word = re.sub('[^\w]','',i).split()
        if len(word)>0:
            for k in word:
                if(k in string.punctuation):
                    word.remove(k)

        else:
            print("Removed Sentence:",i)
            continue
        res = calculate_sentence_embadding(word)
        sent_emb.append(res)
    return (sent_emb)

In [17]:
def calculate_sentence_embadding(word):
    emb_li = []
    for k in word[0][0]:
        emb_vector = emb_index.get(k)
        if emb_vector is not None:
            if len(emb_vector) == 25:
                emb_li.append(list(emb_vector))
    mean_arr = np.array(emb_li)
    mean_arr = np.mean(mean_arr, axis=0)
    return(mean_arr)
            

In [18]:
sent_embadding = embadding_fr_sentence(list_of_sentence)
if len(sent_embadding) > 0:
    cluster = int(np.ceil(len(sent_embadding) ** 0.5))
    kmean = KMeans(init='k-means++',n_clusters=cluster, random_state=0)
    kmean = kmean.fit(sent_embadding)
    avg = []
    closest = []
    for j in range(cluster):
        ix = np.where(kmean.labels_ == j)[0]
        avg.append(np.mean(ix))
    closest,_= pairwise_distances_argmin_min(kmean.cluster_centers_, sent_embadding)
    ordering = sorted(range(cluster), key=lambda k: avg[k])
    summ = "".join(list_of_sentence[closest[ix]] for ix in ordering)
    print(summ)

In the production of SMC (Sheet Moulding Compound), the maturing of the semi-finished product (resin+glass fibre) is of decisive importance.Possible defects due to short maturing and soft semi-finished products are lack of fibre transport, while too long maturing and hard semi-finished products result in incompletely filled components.By adjusting the press parameters such as closing force, closing speed, mould temperature etc., the fluctuations in thickening can normally be compensated.


In [56]:
text

'In the production of SMC (Sheet Moulding Compound), the maturing of the semi-finished product (resin+glass fibre) is of decisive importance. The associated thickening of the material determines the viscosity and thus the quality of the end product. Possible defects due to short maturing and soft semi-finished products are lack of fibre transport, while too long maturing and hard semi-finished products result in incompletely filled components. By adjusting the press parameters such as closing force, closing speed, mould temperature etc., the fluctuations in thickening can normally be compensated. By measuring the flowability/viscosity of the material or by measuring additional parameters during the manufacturing process, the ideal process window for the production of SMC is to be controlled even better.'